# Finary
A fine altenate and potentially novel binary representation. A work in progress.

Surreal binary is standard binary with an alternate value representation. In standard binary the least significant bit is furtherst right and represents the placeholder for 1. The second bit represents a value of 2 and so on, doubling the value of the bit for each bit we add to the size of the binary register.

In surreal binary the value of the bits is a non constant power of 2. The power of the positions is determined by the number of repeating bits at the beginning of its binary representation.

The name "surreal" is derived from the surreal numbers and their connection is described below.

This paper elaborate on these ideas and provides analytic proof using python code samples.

Author: Jeff Anderson

### Binary review
In binary digits represent values of: 1, 2, 4, 8, 16... where the value of a specific number is the sum of the values having digits of "1" in its binary representation:

In [3]:
print('\ndecimal binary')
for n in range(8):    
    print('{:>5}  : {:>3}'.format(n,bin(n)[2:]))
        
print('\n...where each digits is a power of 2\n')
print('decimal binary')
for n in range(8):
    n = 2**n
    print('{:>5}  : {:<}'.format(n,bin(n)[2:]))



decimal binary
    0  :   0
    1  :   1
    2  :  10
    3  :  11
    4  : 100
    5  : 101
    6  : 110
    7  : 111

...where each digits is a power of 2

decimal binary
    1  : 1
    2  : 10
    4  : 100
    8  : 1000
   16  : 10000
   32  : 100000
   64  : 1000000
  128  : 10000000


### Van der Corput sequence
A Van der Corput sequence is the simplest one-dimensional low-discrepancy sequence over the unit interval.

The value of a binary Van der Corput number is the sum of ±1/2 for the first bit, ±1/4 for the second bit and continues to half for subsequent bits. The sign of this value is positive if the bit is set in that position or negative otherwise.

Here is an implementation of a standard binary count on a number represented by this sequence.

In [1]:
from fractions import Fraction

def vandercorput (x):
    n = 0
    p = 0
    for bit in x:
        if bit == '1':
            s = 1
        else:
            s = -1
        p -=1
        n += s * 2**p
    return n

print('\n  binary : count :  decimal : fraction')
for n in range(1,16):
    binary = bin(n)[2:]
    vcorp  = vandercorput(binary)
    dyatic = Fraction(vcorp)

    print('{:>7}  : {:4}  :  {:>6}  :   {:7}'.format(binary,n,vcorp,str(dyatic)))


  binary : count :  decimal : fraction
      1  :    1  :     0.5  :   1/2    
     10  :    2  :    0.25  :   1/4    
     11  :    3  :    0.75  :   3/4    
    100  :    4  :   0.125  :   1/8    
    101  :    5  :   0.375  :   3/8    
    110  :    6  :   0.625  :   5/8    
    111  :    7  :   0.875  :   7/8    
   1000  :    8  :  0.0625  :   1/16   
   1001  :    9  :  0.1875  :   3/16   
   1010  :   10  :  0.3125  :   5/16   
   1011  :   11  :  0.4375  :   7/16   
   1100  :   12  :  0.5625  :   9/16   
   1101  :   13  :  0.6875  :   11/16  
   1110  :   14  :  0.8125  :   13/16  
   1111  :   15  :  0.9375  :   15/16  


<img src="files/image/mapping_4k5.png">

<img src="files/image/mapping_4k3.png">

Notice that the decimal number is splitting the interval between 0 and 1 over at double the precision. The first value 0.5 splits 0 to 1 in half. The next 2 (0.25 and 0.75) split the interval in half twice more. The next four split those intervals in half and then the next eight do the same and so on.

The conversion of the number is + positive when the bit is 1 and - negative when the bit is 0.

For example: 

    9 decimal = 1001 binary
  
    "1001" converted to signs = "+--+"
  
so the number represents declining half fractions using these signs:
  
    = + 1/2 - 1/4 - 1/8 + 1/16
  
    = 3/16 = 0.1875

Notice that although the binary count continues to grow toward infinity, the Van der Corput binary sequence remains bound as a fraction between zero and one.

Closer inspection reveals that the divisions cycle across the unit domain repeat at powers of two while maintaining a binary incremental count. 
Each cycle splits the divisions in half to provide more precision to these divisions of one. 
The division spacing will be the inverse power of two matching the size of the number.

### Negative sequence
But a binary Van der Corput sequences in a computer register is different than written sequence, because a register will have a set bit size. Since the 0 values in a van der corput sequence have value, as negative inverse powers of two, these leading values are significant.

This means that a standard count in a binary computer register will result in a binary van der corput sequence that is different than the sequence previously demonstrated.

Consider these differences:

In [3]:
# in binary these are all equal to the number 1
equivelent_binary = '''
    00000001 
     0000001
      000001 
       00001 
        0001 
         001 
          01 
'''.split()

print('\n Standard Binary : Van der Corput binary\n')
for binary in equivelent_binary:
    print('   {:>12}  :{:>12}'.format(binary,vandercorput(binary)))


 Standard Binary : Van der Corput binary

       00000001  : -0.98828125
        0000001  :  -0.9765625
         000001  :   -0.953125
          00001  :    -0.90625
           0001  :     -0.8125
            001  :      -0.625
             01  :       -0.25


The numbers are not irratic. They are diffrentiated by powers of two.

In [116]:
from math import log
import string
import re
to_power = str.maketrans('-0123456789','⁻⁰¹²³⁴⁵⁶⁷⁸⁹')
print('\n  Van der Corput power values\n')
previous = None
for binary in equivelent_binary:
    vcorp = vandercorput(binary)
    if previous:
        difference = vcorp - previous
        power_diff = (str(int(log(difference,2)))).translate(to_power)
        print(' {:>11} + ({:>11}) = {:<9} = 2{}'.format(vcorp,previous,difference,power_diff))
    previous = vcorp


  Van der Corput power values

 -0.98046875 + (-0.98828125) = 0.0078125 = 2⁻⁷
 -0.96484375 + (-0.98046875) = 0.015625  = 2⁻⁶
 -0.93359375 + (-0.96484375) = 0.03125   = 2⁻⁵
 -0.87109375 + (-0.93359375) = 0.0625    = 2⁻⁴
 -0.74609375 + (-0.87109375) = 0.125     = 2⁻³
 -0.49609375 + (-0.74609375) = 0.25      = 2⁻²
  0.00390625 + (-0.49609375) = 0.5       = 2⁻¹


### Dyatic binary sequence
The dyatic rationals are the fractions with denominators of powers of two.

Below is binary representation of the dyatics modified projection of the Van der Corput sequence as a generator.

In [133]:
def dyatic_binary (r=[Fraction(0)]):
    yield r[0]
    while 1:
        yield r[0]-1
        rn = [r[0]]
        for n in r[1:]:
            mid = (rn[-1]+n)/2
            yield mid
            rn.append(mid)
            rn.append(n)
        yield rn[-1]+1
        r = [rn[0]-1] + rn + [rn[-1]+1]
dyatic_sequence = dyatic_binary()
print('count : dyatic')
for n in range(16):
    dyatic = next(dyatic_sequence)
    print(' {:>3}  : {:>4}'.format(n,str(dyatic)))

count : dyatic
   0  :    0
   1  :   -1
   2  :    1
   3  :   -2
   4  : -1/2
   5  :  1/2
   6  :    2
   7  :   -3
   8  : -3/2
   9  : -3/4
  10  : -1/4
  11  :  1/4
  12  :  3/4
  13  :  3/2
  14  :    3
  15  :   -4


The range between -1 and 1 is evenly quartered, while the remaining values extend aways at powers of two which are also split in suquent iterations of the count.

This results in a value coverage like this:

*image to be attached

### Surreal binary
The surreal binary system is similar to the dyatic sequence, but instead the leading bits represent sums instead of powers.

In essence, the first bits of a surreal binary number are like primitive tally marks. Each bit has a value of one and the total of the number is the total numbers of set bits.

For example:

     b.surreal  value
           1      1
          11      2
         111      3
        1111      4
       11111      5

It would seem that surreal binary can not represent as many numbers as standard binary would allow. It is not the case. Surreal numbers are fractional so the number of values represented will be the same. The domain of the numbers is simply more confined.
 

In [5]:
def surreal_binary (r=[Fraction(0)]):
    yield r[0]
    while 1:
        yield r[0]-1
        rn = [r[0]]
        for n in r[1:]:
            mid = (rn[-1]+n)/2
            yield mid
            rn.append(mid)
            rn.append(n)
        yield rn[-1]+1
        r = [rn[0]-1] + rn + [rn[-1]+1]
surreal_sequence = surreal_binary()
print('count : surreal')
for n in range(32):
    surreal = next(surreal_sequence)
    print(' {:>3}  : {:>4}'.format(n,str(surreal)))

count : surreal
   0  :    0
   1  :   -1
   2  :    1
   3  :   -2
   4  : -1/2
   5  :  1/2
   6  :    2
   7  :   -3
   8  : -3/2
   9  : -3/4
  10  : -1/4
  11  :  1/4
  12  :  3/4
  13  :  3/2
  14  :    3
  15  :   -4
  16  : -5/2
  17  : -7/4
  18  : -5/4
  19  : -7/8
  20  : -5/8
  21  : -3/8
  22  : -1/8
  23  :  1/8
  24  :  3/8
  25  :  5/8
  26  :  7/8
  27  :  5/4
  28  :  7/4
  29  :  5/2
  30  :    4
  31  :   -5


<img src="files/image/surreal_map_sm1.png">

<img src="files/image/surreal_map_sm2.png">

# Features

    0111111111111...1 is closest to -0
    
    1000000000000...0 is closest to +0
    
 and on the large scale...
    
    1111111111111...1, nearest to +inf
    
    0000000000000...0, nearest to -inf
    
 and furthermore, the closest number on either side of x is:
 
     x01111111111...1111, nearest to x on negative side of x
     
     x10000000000...0000, nearest to x on positive side of x
 
 these infinite strings are both "1":
 
     101111111111....1111
 
     110000000000....0000
 
 but different for any finite length. For example, these are different deven thought their continuation is the same number (1):
 
     10111111111  ≠  11000000000
     
 because those numbers represent: 1 ± 1/1024. 
 They are close.
     
 other infinite representations:
 
     1010101010101010101... = 3

     1001001001001001001... = e


### Conclusion
Finite surreal numbers have a binary representation.
They have these interesting properties:

- all possible values for a register average to zero
- zero itself is the most mixed state
- powers beyond the register size can be represented
- divisions remain even despite the domain spread as register size increases
- binary surreal numbers have only one representation per value
- binary surreal numbers do not have a sign bit and instead push polarity into the whole representation
- negative and positive values are perfect reflections.
- only infinite values have duplicate representations
- there is no value for zero, but instead their is a negative and positive approximation: 
     

### Funciton tools
Below are a couple of functions that create power binary and left justified binary from an integer or floating point value.

In [16]:
from math import e, pi, log
DIGITS = 21
# I don't think this is work quite right...
def to_power_binary (n,precision=DIGITS):
    if not n:
        return ''
    sign = n/abs(n)
    v,n,p = '',-n,-1
    while (abs(n)>1 and len(v) <= precision):
        v += '1'
        n /= 2
        p += 1
    while (n and len(v) <= precision):
        if abs(n + 2**p) > abs(n - 2**p):
            v += '1'
            n -= 2**p
        else:
            v += '0'
            n += 2**p
        p += 1
    return v
for x in (1,0,-1,2,3,4,5,1/2,1/3,1/4,1/5,pi,e,log(2)):
    print('{:>20} : {}'.format(x,to_power_binary(x)))

                   1 : 0010101010101010101010
                   0 : 
                  -1 : 1101010101010101010101
                   2 : 10
                   3 : 1101010101010101010101
                   4 : 1101010101010101010101
                   5 : 1110101010101010101010
                 0.5 : 0
  0.3333333333333333 : 0101010101010101010101
                0.25 : 0101010101010101010101
                 0.2 : 0101010101010101010101
   3.141592653589793 : 1101010101010101010101
   2.718281828459045 : 1101010101010101010101
  0.6931471805599453 : 0010101010101010101010


In [17]:
from math import floor, sqrt, log, e, pi
# base can be anything (imo) between 1 and 2: 1/e,i phi = (sqrt(5)+1)/2
base = 2
bits = 40
# 1 and 2 are the same because it is right padded with 0's. That's not right.
def left_binary (binary):
    return bin(~int(x*base**(bits-floor(log(x,base))-base)+base**bits))[2:]

print('\nleft justified binary\n')
print('using a base of',base)
print('and a register having {} bits'.format(bits))

for x in range(1,20):
    r=39
    print('{:>3} : {}'.format(x,left_binary(x)))


left justified binary

using a base of 2
and a register having 40 bits
  1 : b10100000000000000000000000000000000000001
  2 : b10100000000000000000000000000000000000001
  3 : b10110000000000000000000000000000000000001
  4 : b10100000000000000000000000000000000000001
  5 : b10101000000000000000000000000000000000001
  6 : b10110000000000000000000000000000000000001
  7 : b10111000000000000000000000000000000000001
  8 : b10100000000000000000000000000000000000001
  9 : b10100100000000000000000000000000000000001
 10 : b10101000000000000000000000000000000000001
 11 : b10101100000000000000000000000000000000001
 12 : b10110000000000000000000000000000000000001
 13 : b10110100000000000000000000000000000000001
 14 : b10111000000000000000000000000000000000001
 15 : b10111100000000000000000000000000000000001
 16 : b10100000000000000000000000000000000000001
 17 : b10100010000000000000000000000000000000001
 18 : b10100100000000000000000000000000000000001
 19 : b101001100000000000000000000000000000000